# Исследование надёжности заемщиков

Заказчиком является кредитный отдел банка.

Требуется определить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

Входные данные: статистика о платёжеспособности клиентов.

Результаты исследования предполагается использовать при построении модели кредитного скоринга.

### Описание данных
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

## Шаг 1. Обзор данных

### Откроем таблицу и изучим общую информацию о данных

In [1]:
import pandas as pd
try:
    df = pd.read_csv('datasets/data_05.csv') # locally hosted datafile
except:
    df = pd.read_csv('/datasets/data.csv') # Yandex-hosted datafile
df.info()
display(df.describe())
display(df.head(5))
display(df.tail(5))
print('income_type', df['income_type'].unique())
print('education_id', df['education_id'].unique())
print('education', df['education'].unique())
print('family_status_id', df['family_status_id'].unique())
print('family_status', df['family_status'].unique())
print('gender', df['gender'].unique())
print('purpose', df['purpose'].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


income_type ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
education_id [0 1 2 3 4]
education ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
family_status_id [0 1 2 3 4]
family_status ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
gender ['F' 'M' 'XNA']
purpose ['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'с

### Выводы по шагу 1
- Имена колонок укладываются в нотацию Snake Case. Смысл колонок, на первый взгляд, не всегда точно соответсвует данным в них.
- Колонка days_employed местами содержит странные значения. Не похожие на дни, т.к. в этом случае срок занятости у максимально занятого клиента составит 1100 лет. Первая идея - что это минуты или секунды с дня выхода на работу или увольнения (минусы). Необходим анализ данных на их структуру и отклонения.
- в колонке purpose много неявных дублей, нет дедуплицированного словаря.
- группа колонок education(id) содержит разные строковые представления одной категории. что порождает вопросы к источнику, из которого это выгружали. И необходимость дедупликации.
- Колонки days_employed и total_income содержат пропуски (19351 строк вместо 21525 ==> 2174 строк с пропусками)
- Так как число пропусков у двух колонок идентично, стоит проверить гипотезу, что они пропущены в одних и тех же строках (например, при объединении данных из разных систем). Это может быть полезно при анализе других искажений данных.
- У кого-то из клиентов в датасете заявлено 20 детей. Просто посмотреть на этого героя. Также интересно значение -1.
- В поле gender есть значение 'XNA'. Разобраться, может ли это помешать анализу.

## Исправление пропусков и аномалий

### Шаг 2.1 Заполнение пропусков
В двух столбцах есть пропущенные значения, необходимо их найти и заполнить медианным значением по столбцу

#### Проверяем гипотезу, что пропуски обеих колонок находятся в одних и тех же строках

In [2]:
print('Число строк с пропусками:', df.loc[(df['days_employed'].isna()) & (df['total_income'].isna())]['children'].count())
print('Число строк с пропусками days_employed:', df.loc[df['total_income'].isna()]['children'].count())
print('Число строк с пропусками total_income:', df.loc[df['total_income'].isna()]['children'].count())

Число строк с пропусками: 2174
Число строк с пропусками days_employed: 2174
Число строк с пропусками total_income: 2174


<b>Вывод:</b> все пропуски обеих колонок находятся в одних и тех же строках

#### Заполняем пропуски

In [3]:
days_employed_median = df['days_employed'].median()
total_income_median = df['total_income'].median()
print("Медиана для days_employed:", days_employed_median)
print("Медиана для total_income:", total_income_median)
print()

df['days_employed'] = df['days_employed'].fillna(days_employed_median)
df['total_income'] = df['total_income'].fillna(total_income_median)

df.info()
display(df.describe())
display(df.head(15))

Медиана для days_employed: -1203.369528770489
Медиана для total_income: 145017.93753253992

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,56557.335698,43.293380,0.817236,0.972544,0.080883,1.651595e+05
std,1.381587,134922.319298,12.574584,0.548138,1.420324,0.272661,9.786607e+04
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2518.168900,33.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-385.106616,53.000000,1.000000,1.000000,0.000000,1.955436e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


#### Выводы по шагу 2.1
- В столбцах 'days_employed' и 'total_income' выявлены пропущенные значения с типом NaN, они заполнены медианным значением по столбцу.
- Данные столбцы отражают "количество дней занятости" и "суммарный доход".
- Доля пропущенных значений по обеим колонкам составляет 10,09% (2174 пропуска на 21525 строк данных).
- Возможные причины появления пропусков в данных: слияние данных из разных истоников, использование некорректно работающего АРМ при заполнении пропущенных данных.
- Почему заполнить пропуски медианным значением — лучшее решение для количественных переменных? Потому что использование каких-либо "магических цифр", нулей или среднего значения сильнее исказит данные выборки. До тех пор, пока мы не обладаем точной бизнес-информацией о кластеризации и о том, что делать с пропусками (например, указывать какой-то специфический доход для категории 'пенсионеров'),  использование общего медианного значения оправдано.

### Шаг 2.2 Проверка данных на аномалии и исправления.

Обработаем значения в столбцах с аномалиями и опишем возможные причины появления таких данных.

#### Обработка значения колонки 'children' == -1

данное значение встречается в данных 47 раз и похоже на "магическое значение". Так как это не категория, до выяснения логики его заполнения удалим эти строки.

In [4]:
#df.loc[df['children'] == -1, 'children'] # = 0
df = df.loc[df['children'] != -1]
display(df.loc[df['children'] == -1])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


#### Проверка записи с количеством детей = 20

76 записей, не похоже на опечатку, но похоже на системную ошибку, т.к. ближайшее заполненное количество детей - 5. Удалим эти данные как "выброс".

In [5]:
df = df.loc[df['children'] != 20]
display(df.loc[df['children'] == 20])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


#### Обработка значения колонки 'days_employed' < 0
    
Данное значение встречается в данных 15869 раз. Предположительно, отрицательное значение вызвано утратой работы заявителем и отсчитывается с дня такой утраты.
Так как отрицательное значение не имеет вещественного смысла, примем его равным 0 для безработных.

In [6]:
df.loc[df['days_employed'] < 0, 'days_employed'] = 0
display(df.loc[df['days_employed'] < 0])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


#### Проверка "неопределённого" значения поля 'gender'
всего одна запись.
Поскольку аналитические задачи (вопросы проекта) не увязаны с полом, его можно проигнорировать

In [7]:
display(df.loc[df['gender'] == 'XNA'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,0.0,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


#### Анализ столбца dob_years
Сто записей с возрастом 0 лет. Объём данных несущественен.
Поскольку аналитические задачи (вопросы проекта) не увязаны с возрастом, его можно проигнорировать.

In [8]:
print('dob_years', df.sort_values('dob_years')['dob_years'].unique())
display(df.loc[df['dob_years'] == 0])

dob_years [ 0 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41
 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65
 66 67 68 69 70 71 72 73 74 75]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,0.000000,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,0.000000,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,0.000000,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,0.000000,0,среднее,1,женат / замужем,0,F,сотрудник,0,145017.937533,жилье
20462,0,338734.868540,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,покупка своего жилья
20577,0,331741.271455,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,недвижимость
21179,2,0.000000,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости


#### Выводы по шагу 2.2
- Удалены 47 и 76 записей, имеющие в столбце 'children' значения -1 и 20. В идеале необходимо запросить у источника выгрузки смысл таких "магических значений".
- Столбец 'days_employed' оставлен без отрицательных значений. Также необходимо прояснить у источника логику его заполнения. Пока что все 'безработные' получают 0 рабочих дней.
- В столбцах "пол" и "возраст" (dob_years) выявлены незначительные аномалии. Та как эти данные не используются в анализе, они сохранены.
- В задании нигде не фигурируют странные для "дней" значения столбца 'days_employed', поэтому они не анализируются. Но это не отменяет подозрение на то, что в колонке хранятся данные с какой-то иной детализацией или в ином формате. В учебной ситуации, при поиске закономерностей без привязки к конкретным датам и срокам, формат не критичен.

### Шаг 2.3. Изменение типов данных.
#### Заменим вещественный тип данных в столбце total_income на целочисленный.

In [9]:
# смена типа на целочисленный суммы общего дохода 'total_income'
df['total_income'] = df['total_income'].astype('int')

# смена типа на целочисленный для 'days_employed'. Даже если это секунды, их нет смысла хранить в формате вещественных чисел.
df['days_employed'] = df['days_employed'].astype('int')

# проверка внесённых изменений
df.info()
display(df.describe())
display(df.head(5))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21402 non-null  int64 
 1   days_employed     21402 non-null  int64 
 2   dob_years         21402 non-null  int64 
 3   education         21402 non-null  object
 4   education_id      21402 non-null  int64 
 5   family_status     21402 non-null  object
 6   family_status_id  21402 non-null  int64 
 7   gender            21402 non-null  object
 8   income_type       21402 non-null  object
 9   debt              21402 non-null  int64 
 10  total_income      21402 non-null  int64 
 11  purpose           21402 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21402.000000,21402.000000,21402.000000,21402.000000,21402.000000,21402.000000,2.140200e+04
mean,0.473180,58517.995935,43.300206,0.817120,0.973133,0.080927,1.651823e+05
std,0.751837,134195.035251,12.579055,0.548741,1.420536,0.272729,9.799193e+04
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.066700e+04
25%,0.000000,0.000000,33.000000,1.000000,0.000000,0.000000,1.077160e+05
50%,0.000000,0.000000,42.000000,1.000000,0.000000,0.000000,1.450170e+05
75%,1.000000,0.000000,53.000000,1.000000,1.000000,0.000000,1.955722e+05
max,5.000000,401755.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,0,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,0,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,0,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


#### Выводы по шагу 2.3
- Проведена смена типа с вещественного на целочисленный для колонок 'total_income' и 'days_employed'. Первая - по условию задания, вторая - по соображениям сути хранимого в колонке явления.

### Шаг 2.4. Удаление дубликатов.
#### Удаление полных дубликатов и нормализация неявных.
В столбце education есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв.

Приведём к одному регистру схожие значения столбца education. Проверим остальные столбцы.

In [10]:
# перевод в нижний регистр всех значений колонки 'education'
df['education'] = df['education'].str.lower()

#проверка изменений - неявных дублей 'education' нет
print('education', df['education'].unique())
display(df.head(5))


# Датасет содержит потенциальные дубли. Характер представленных данных не позволяет определить, являются ли данные дублями или это разные "похожие" клиенты.
# В рамках учебной ситуации дедупликация будет проведена. 71 запись --> 0 записей
print('Число потенциальных дубликатов строк:', df.duplicated().sum(), '\n')
display(df[df.duplicated()])
df = df.drop_duplicates().reset_index(drop=True)
print('Число потенциальных дубликатов строк (после обработки):', df.duplicated().sum(), '\n')

education ['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,0,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,0,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,0,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Число потенциальных дубликатов строк: 71 



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,0,41,среднее,1,женат / замужем,0,F,сотрудник,0,145017,покупка жилья для семьи
3290,0,0,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу
4182,1,0,34,высшее,0,гражданский брак,1,F,сотрудник,0,145017,свадьба
4851,0,0,60,среднее,1,гражданский брак,1,F,пенсионер,0,145017,свадьба
5557,0,0,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,0,64,среднее,1,женат / замужем,0,F,пенсионер,0,145017,дополнительное образование
21032,0,0,60,среднее,1,женат / замужем,0,F,пенсионер,0,145017,заняться образованием
21132,0,0,47,среднее,1,женат / замужем,0,F,сотрудник,0,145017,ремонт жилью
21281,1,0,30,высшее,0,женат / замужем,0,F,сотрудник,0,145017,покупка коммерческой недвижимости


Число потенциальных дубликатов строк (после обработки): 0 



#### Выводы по шагу 2.4
- С помощью перевода в нижний регистр удалены неявные дубли в колонке 'education'.
- Как показал анализ данных в п.1, колонки 'income_type', 'family_status', 'gender' дублей не содержат.
- Колонка 'purpose' будет обрабатываться позже функцией классификации по вхождениям слов.
- Метод для поиска и удаления дубликатов в данных: были удалены полные дубли строк с пересчётом индекса, а также заменены неявные дубли значений колонки 'education'
- Возможные причины появления дубликатов: некорректно написанный запрос к данным, слияние данных из разных источников, ошибки работы АРМ в месте ввода или ETL-процедур.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.
#### Создадим два новых датафрейма словарей
В которых:
* каждому уникальному значению из education соответствует уникальное значение education_id — в первом;
* каждому уникальному значению из family_status соответствует уникальное значение family_status_id — во втором.

Удалим из исходного датафрейма столбцы education и family_status, оставив только их идентификаторы: education_id и family_status_id.

In [11]:
df_education = df.loc[:,['education_id','education']].drop_duplicates().reset_index(drop=True)
display(df_education)

df_family_status = df.loc[:,['family_status_id','family_status']].drop_duplicates().reset_index(drop=True)
display(df_family_status)


df = df.drop(columns=['education', 'family_status'])
display(df.head(5))

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,0,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,0,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,0,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,0,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


#### Выводы по шагу 2.5
- Созданы датафреймы-"словари" для семейного статуса и образования.
- Удалены колонки с текстовыми представлениями данных статусов из основного датафрейма.

### Шаг 2.6. Категоризация дохода.

In [12]:
# На основании диапазонов, указанных ниже, создадим столбец total_income_category с категориями:
#   0–30000 — 'E';
#   30001–50000 — 'D';
#   50001–200000 — 'C';
#   200001–1000000 — 'B';
#   1000001 и выше — 'A'.
# Например, кредитополучателю с доходом 25000 нужно назначить категорию 'E', а клиенту, получающему 235000, — 'B'.

def set_total_income_category(row):
    total_income = row['total_income']
    if total_income < 30000:
        return 'E'
    elif (total_income >= 30001) & (total_income <= 50000):
        return 'D'
    elif (total_income >= 50001) & (total_income <= 200000):
        return 'C'
    elif (total_income >= 200001) & (total_income <= 1000000):
        return 'B'
    else: # 1000001 и выше — 'A'
        return 'A'
    # --- вариант передачи некорректного значения в учебной ситуации не обрабатываем

df['total_income_category'] = df.apply(set_total_income_category, axis=1)
display(df.head(10))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,0,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,0,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,0,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,0,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,0,27,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,0,43,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,0,50,1,0,M,сотрудник,0,135823,образование,C
8,2,0,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,0,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


### Шаг 2.7. Категоризация целей кредита.

In [13]:
# Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который входят следующие категории:
#   'операции с автомобилем',
#   'операции с недвижимостью',
#   'проведение свадьбы',
#   'получение образования'.
# Например, если в столбце purpose находится подстрока 'на покупку автомобиля', то в столбце purpose_category должна появиться строка 'операции с автомобилем'.

def set_purpose_category(row):
    purpose = row['purpose']
    if ('автом' in purpose ) | ('машин' in purpose ):
        return 'операции с автомобилем'
    elif ('недвиж' in purpose ) | ('жиль' in purpose ):
        return 'операции с недвижимостью'
    elif ('свадьб' in purpose ) | ('свадеб' in purpose ):
        return 'проведение свадьбы'
    elif ('образов' in purpose ) | ('обучен' in purpose ):
        return 'получение образования'
    else: # --- для отладки при несовпадении возвращаем пустую строку
        return ''

df['purpose_category'] = df.apply(set_purpose_category, axis=1)
display(df.head(10))

# проверяем, что нераспределённых значений не осталось
display(df.loc[df['purpose_category'] == ''])

# проверяем, что нет некорректных распределений
display(df.loc[:,['purpose','purpose_category']].drop_duplicates().reset_index(drop=True))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,0,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,0,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,0,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,0,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,0,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,0,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,0,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,0,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,0,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category


,purpose,purpose_category
0,покупка жилья,операции с недвижимостью
1,приобретение автомобиля,операции с автомобилем
2,дополнительное образование,получение образования
3,сыграть свадьбу,проведение свадьбы
4,операции с жильем,операции с недвижимостью
5,образование,получение образования
6,на проведение свадьбы,проведение свадьбы
7,покупка жилья для семьи,операции с недвижимостью
8,покупка недвижимости,операции с недвижимостью
9,покупка коммерческой недвижимости,операции с недвижимостью


### Ответы на вопросы.

#### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [14]:
# определим общее количество должников в отношении к числу детей
df_pivot_01 = df.groupby(['children', 'debt']).agg({'children':'count'})
display(df_pivot_01)

# вычислим процент должников для каждого количества детей
df_pivot_01_total = df.groupby(['children']).agg({'debt': 'mean'})
display(df_pivot_01_total)

children
children debt          
0        0        13028
         1         1063
1        0         4364
         1          444
2        0         1858
         1          194
3        0          303
         1           27
4        0           37
         1            4
5        0            9

,debt
children,
0,0.075438
1,0.092346
2,0.094542
3,0.081818
4,0.097561
5,0.000000


##### Вывод 1:
Между количеством детей и возвратом кредита наблюдается определённая зависимость:
- Заёмщики без детей возвращают кредиты в срок на 1,5% чаще остальных.
- Заёмщики с 3 детьми чуть более ответсвенны, чем с 1,2 и 4 детьми.
- Выборка заёмщиков с 5 детьми слишком мала чтобы делать выводы.
- В целом, заёмщики с детьми возвращают кредиты хуже чем заёмщики без детей.

#### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [15]:
# определим общее количество должников в отношении к семейному положению
df_pivot_02 = df.groupby(['family_status_id', 'debt']).agg({'family_status_id':'count'})
display(df_pivot_02)

# вычислим процент должников для каждого класса семейного положения
df_pivot_02_total = df.groupby(['family_status_id']).agg({'debt': 'mean'}).merge(df_family_status, on='family_status_id', how='left').sort_values('debt')
display(df_pivot_02_total)

family_status_id
family_status_id debt                  
0                0                11334
                 1                  927
1                0                 3749
                 1                  385
2                0                  888
                 1                   63
3                0                 1105
                 1                   84
4                0                 2523
                 1                  273

,family_status_id,debt,family_status
2,2,0.066246,вдовец / вдова
3,3,0.070648,в разводе
0,0,0.075606,женат / замужем
1,1,0.093130,гражданский брак
4,4,0.097639,Не женат / не замужем


##### Ответ 2:
Зависимость между семейным положением и возвратом кредита в срок имеется:
- наиболее "надёжными" являются вдовцы и вдовы, после них - разведённые и, наконец, состящие в официальном браке.
- наиболее рисковыми (с разницей в 2+%) являются заёмщики, находящиеся в гражданском браке и не входившие в брак вовсе.


#### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [16]:
# определим общее количество должников в отношении к уровню дохода
df_pivot_03 = df.groupby(['total_income_category', 'debt']).agg({'total_income_category':'count'})
display(df_pivot_03)

# вычислим процент должников для каждого класса семейного положения
df_pivot_03_total = df.groupby(['total_income_category']).agg({'debt': 'mean'}).sort_values('debt')
display(df_pivot_03_total)

total_income_category
total_income_category debt                       
A                     0                        23
                      1                         2
B                     0                      4659
                      1                       354
C                     0                     14569
                      1                      1353
D                     0                       328
                      1                        21
E                     0                        20
                      1                         2

,debt
total_income_category,
D,0.060172
B,0.070616
A,0.080000
C,0.084977
E,0.090909


##### Ответ 3:
Зависимость между уровнем дохода и возвратом кредита в срок имеется:
- Наиболее "безопасными" являются клиенты с относительно скромными уровнями дохода 30001–50000 (категория D).
- достаточно надёжными являются клиенты с доходом "выше среднего: 200001–1000000 (категория B).
- Клиенты со сверхвысоким доходом (1000001 и выше, категория A) и клиенты со средним доходом (50001–200000, категория C) демонстрируют умеренный риск.
- Наименее надёжными являются заёмщики с самым низким уровнем дохода - 0–30000 (категория E).


#### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [17]:
# определим общее количество должников в отношении к семейному положению
df_pivot_04 = df.groupby(['purpose_category', 'debt']).agg({'purpose_category':'count'})
display(df_pivot_04)

# вычислим процент должников для каждого класса семейного положения
df_pivot_04_total = df.groupby(['purpose_category']).agg({'debt': 'mean'}).sort_values('debt')
display(df_pivot_04_total)

purpose_category
purpose_category         debt                  
операции с автомобилем   0                 3879
                         1                  400
операции с недвижимостью 0                 9971
                         1                  780
получение образования    0                 3619
                         1                  369
проведение свадьбы       0                 2130
                         1                  183

,debt
purpose_category,
операции с недвижимостью,0.072551
проведение свадьбы,0.079118
получение образования,0.092528
операции с автомобилем,0.093480


##### Ответ 4:
Разные цели кредита влияют на его возврат в срок следующим образом:
- фактически, наблюдается разделение на две категории: в "надёжную" можно включить операции с недвижимостью и свадьбы. К ненадёжным можно отнести сферу образования и операции с автомобилем.
- Сами явления, стоящие за цказанными целями, также подтверждают это распределение: брак и недвижимость планируются более размеренно, нежели обучение и приобретение движимого имущества.

#### Вывод 1:

Очевидно, что все четыре разреза имеют влияние на риски по возврату кредитов. и, при их объединении, могут давать существенно бОльшие отклонения в оценках.
Это весьма полезный и рабочий инструмент.

## Общий вывод:

* Проведён предварительный анализ, очистка и дедупликация данных.
    * Выявлен неоднозначный формат данных в колонке days_employed, не соответсвующий собственно дням.
    * Выявлена портебность в дедупликации колонок purpose и education(id)
    * Выявлены и исправлены пропуски в колонках days_employed и total_income (10,09% от общего объёма данных)
    * Выявлены и исправлены выбросы в колонке "число детей"
    * В данных выявлен неопределённый пол
* Удалены дубликаты данных и созданы словари данных по образованию и семейному положению
* Проведена классификация клиентов по уровню дохода и целям получения кредита
* Проведён анализ зависимости между возвратом кредита в срок и следующими параметрами:
    * Количество детей.
        * Заёмщики без детей возвращают кредиты в срок на 1,5% чаще остальных.
        * Заёмщики с 3 детьми чуть более ответсвенны, чем с 1,2 и 4 детьми.
        * Выборка заёмщиков с 5 детьми слишком мала чтобы делать выводы.
        * В целом, заёмщики с детьми возвращают кредиты хуже чем заёмщики без детей.
    * Уровень дохода
        * Наименьший риск - по клиентоам с относительно скромными уровнями дохода 30001–50000 (категория D).
        * Достаточно надёжными являются клиенты с доходом "выше среднего: 200001–1000000 (категория B).
        * Клиенты со сверхвысоким доходом (1000001 и выше, категория A) и клиенты со средним доходом (50001–200000, категория C) демонстрируют умеренный риск.
        * Наименее надёжными являются заёмщики с самым низким уровнем дохода - 0–30000 (категория E).
    * Разные цели кредита.
        * Наиболее надёжными являются операции с недвижимостью и свадьбы.
        * Менее надёжны сфера образования и операции с автомобилем. 
    * Семейное положение
        * Наиболее "надёжными" являются вдовцы и вдовы, после них - разведённые и, наконец, состящие в официальном браке.
        * Более рисковыми являются заёмщики, находящиеся в гражданском браке и не входившие в брак вовсе.
    
Все проанализированные разрезы показали наличие влияния на риски по возврату кредитов.